# Gas Discharge & Stellar Feedback-driven Outflows (Part 3 /3):
## Ram Pressure, Accretion Times, Distributions
`-- Leo Lonzarich, 17 November 2021 // [Updated 28-11-21] --`


### Table of Contents
- [Ram Pressure Calculations for Discharged Gas](#ram)
    - [Identifying Ram Pressures](#ramdis)


### General Comments on the Contents
- Repeated pids that turn up in the data sets below (e.g. 'ejected,' 'expelled,' 'discharged') should mean that the same particle has been accreted and reejected more than once --- hence its reoccurance in the set.


- The datasets used in this notebook: (note that each logs the properties for their respective particles for the period proceeding the differentiating event)

    - **Ejected/Expelled Datasets of pulled from Akins:**
        - *Ejected*: Gas particles that have been removed from their respective satellite galaxy's disk to the halo.
        - *Expelled*: Gas particles (within the disk or halo of their respective satellite galaxy) that have been removed beyond the virial radius of their satellite; that is, all gas particles removed to the field.
        
    - **Discharge Datasets From Lonzarich:**
        - *Discharged*: Gas particles that have been removed from the disk of their respective satellite galaxy. This includes both gas moved to halo, and that moved beyond the virial radius.
        - *Predischarged*: Contains the same particles in *Discharged*, but with properties of each particle logged prior to discharge.
        - *Accreted*: Gas particles in the halo or beyond the virial radius of their respective satellite as the result of a discharge event, but that were accreted back onto the satellite's disk.
        - *Heated*: Gas particles in *Discharged* that were also supernova-heated at time of discharge (i.e., their 'cooling' parameter was turned off).
        - *Preheated*: Contains the same particles in *Heated*, but with properties of each particle logged prior to discharge.

    
    
- Units for key values used in this notebook:
    - *Time* is in Gigayears $(Gyrs)$.
    - *Distances* in kiloparsecs $(kpc)$.
    - *Velocities* in kilometers per second $(km \ s^{-1})$.
    - *Masses* in solar masses $(Msol)$.

In [1]:
import numpy as np
import pylab
import pynbody
import pynbody.plot.sph as sph
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd

from astropy import units
from astropy import constants as c

import warnings
warnings.filterwarnings("ignore")

from compiler import *
from analysis import * 
from ramdischarge import *

In [2]:
path = f'{rootPath}Stellar_Feedback_Code/SNeData/discharged_particles.hdf5'
with pd.HDFStore(path) as hdf:
    keys = np.array([h[1:] for h in hdf.keys()])

# listing dwarf galaxies of simulations that are viable for study (gaseous, satellite, etc).
print(len(keys), 'Available Keys:', *keys)

19 Available Keys: h148_13 h148_278 h148_28 h148_283 h148_329 h148_37 h148_45 h148_68 h148_80 h229_20 h229_22 h229_23 h229_27 h229_55 h242_24 h242_41 h242_80 h329_137 h329_33


## Ram Pressure Stripping  <a class="anchor" id="ram"></a>

For this part, we use ram stripping data collected for each discharged gas particle to see if we can flesh out any differences in the exit angles between discharged and heated gas not visible in initial exit angle plots (see Part 1 of this notebook for reference).
____
____



### Picking out Ram Pressures for Discharged Particles <a class="anchor" id="ramdis"></a>

In [3]:
#ejected, cooled, expelled, accreted = read_all_ejected_expelled()
predischarged, discharged, accreted, preheated, heated = read_all_discharged()

h148_13 h148_278 h148_28 h148_283 h148_329 h148_37 h148_45 h148_68 h148_80 h229_20 h229_22 h229_23 h229_27 h229_55 h242_24 h242_41 h242_80 h329_137 h329_33
> Returning (predischarged, discharged, accreted, preheated, heated) for all available satellites <


In [2]:
pram = read_all_ram_pressure()

1 h148_13 h148_278 h148_28 h148_283 h148_329 h148_37 h148_45 h148_68 h148_80 h229_20 h229_22 h229_23 h229_27 h229_55 h242_24 h242_41 h242_80 h329_137 h329_33
> Returning (predischarged, discharged, accreted, preheated, heated) for all available satellites <


KeyError: 'time'

In [6]:
# pulling out a subset of 'discharged' containing those discharged particles with pids that match the pids of those
# in 'accreted'; calling it 'accdis'; the masking technique.

# Ids of preheated
x = np.array(pram['pid'])
# Ids of discharged, SNe heated gas
y = np.array(discharged['pid'])

index = np.argsort(x)
sorted_x = x[index] # Sorted list of ids pre discharged                        
sorted_index = np.searchsorted(sorted_x, y)
yindex = np.take(index, sorted_index, mode="clip")
mask = x[yindex] != y
result = np.ma.array(yindex, mask=mask)
pram_all = discharged.iloc[result[~result.mask].data]

KeyError: 'pid'